# Time to have fun!

We have finished the theoretical lecture. You can now go over the older notebooks, finishing tasks or understanding better the subjects.

You can also try some more advanced stuff froma working python library, which is not still public: `qcomps!`

With this library you can run (almost) any qiskit circuit on an mps simulator

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from qiskit import QuantumCircuit

from qcomps import run_simulation
from qcomps import QCConvergenceParameters

In [ ]:
# Build the circuit you are interested in. Even a large number of qubits is allowed!
nn = 50
qc = QuantumCircuit(nn)
qc.h(0)
for ii in range(nn-1):
    qc.cx(ii, ii+1)

In [ ]:
# Run it on the mps simulator of qcomps
conv_params = QCConvergenceParameters(max_bond_dimension=100)
result = run_simulation(qc, convergence_parameters=conv_params,
    do_statevector=False, nshots=1024, save_mps='F', do_entanglement=True, approach='PY')

## Measurable quantities

here we report all the quantities that can be measured at the end of a simulation.

### Measures

With measures we denote the projective measurements performed at the end of a simulation.
We perform *nshots* such measurements. These quantity can be accessed from the
:py:func:`simulation_results.measures`. The format of the measurement is a dictionary,
where the keys are the measured states and the values the number of times that state has
been measured. The format of the state is conditioned by the *local_dim* of the degrees of freedom:

* if *local_dim*=2, then the single-site states are not separated by a comma, i.e.
    a state measured in |000> will have a key `'000'`;
* if *local_dim*>2, then the single-site states **are** separated by a comma, i.e.
    a state measured in |000> will have a key `'0,0,0'`. This is to ensure that the
    result stays readable even if the measured state is $\geq 10$.

**We use the same notation of qiskit, which is opposite to the usual notation used in theory**.
This means that the sites are not reported as $|s_1s_2\dots s_n\rangle$,
but as $|s_ns_{n-1}\dots s_1\rangle$, to be consistent with the common notation
in bitstrings.

```python
res = run_simulation(GHZ_qc, bond_dim=4, nshots=1024)
measures = res.measure
for key in measures.keys():
    print(key, ': ', str( measures[key] ) )

>>> '000' : 510
>>> '111' : 514
```


### Statevector

The statevector returns the full statevector of the system with $2^n$ elements.
It SHOULD NOT be addressed when the number of sites is $n\geq 30$, and it is
enabled by the *do_statevector* parameter. It is nevertheless useful for comparison
and testing when the number of sites is small.
It can be addressed using `simulation_results.statevector`. Notice that,
as stated in the prievous section, the notation is different from the one commonly
used in theory, and so also the coefficients of the vector are not the same.

### Computational time

It is simply the time taken by the evolution.

### Singular values cut

The singular values cut are an indication of the approximation taking place during the
simulation. Indeed, at each application of two-qubit there is an approximation going on,
subject to the *bond_dim* $\chi$ and the *cut_ratio* $\epsilon$ parameters.
Based on the *singval_mode* the singular values are reported:

* *singval_mode=M*, at each two-site operator we save the maximum of the singular values cut
* *singval_mode=C*, at each two-site operator we save the **sum** of the singular values cut

We recall that, given a decreasing-ordered vector of singular values
$S=(s_1, s_2, \dots, s_n)$ we truncate all the singular values:

$$
s_i \; \mbox{ is truncated }\quad \mbox{if} \; i>\chi \; \mbox{ or } \;
\frac{s_i}{s_1}<\epsilon
$$

### Entanglement

The bond entanglement entropy across each bond at the end of the simulation. It is
a $n-1$ dimensional vector of non-negative values, defined as:

$$
    S_V= - \sum_{i=1}^\chi s_i^2 \log(s_i^2)
$$


### MPS state and file size

The MPS structure with row-major tensors, to be used in python for further analysis
or as an input state for following simulations. It is available only if the parameter
*save_mps* is set to `F`. We also provide the file size of the MPS state, to have
a measure of how big is the MPS. This size is available when *save_mps* is `F` or `U`.

### Observables measurements

The observables measured at the end of the simulation, formatted as dict. The keys
are the observables name, while the value their measurements. For further information
see the :doc:`chapters/observables` page.


In [ ]:
# Look at the interesting quantities!
measures = result.measures
print('==== MEASURES ====')
for key in measures:
    print(key, measures[key])

time = result.computational_time
print('\n==== TIME ====')
print(time)

entanglement = result.entanglement
print('\n==== ENTANGLEMENT ====')
print(entanglement)

state = result.mps

size = result.sizeof
print('\n==== SIZE ====')
print(size)

statevector = result.statevector
